In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_preprocessing import datetime_processing, userinput_processing, holiday_processing

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score

In [153]:
import os
os.chdir('..')
os.getcwd()

'/Users/varunvenkatesh/Documents/GitHub'

In [154]:
df = pd.read_parquet('data/sessions/caltech/').reset_index(drop=True)
df = datetime_processing(df)
df = userinput_processing(df)
df = holiday_processing(df)

FileNotFoundError: [Errno 2] No such file or directory: 'data/sessions/caltech/'

In [ ]:
df.head()

In [ ]:
def get_results(y_test, prediction):
    cm = confusion_matrix(y_test,prediction)
    make_classification_plot(cm)
    
    results = {'tpr': cm[1, 1]/np.sum(cm[1]),
               'fpr': cm[0,1]/np.sum(cm[0]),
               'accuracy': accuracy_score(y_test, prediction),
               'precision': precision_score(y_test, prediction),
               'recall': recall_score(y_test, prediction),
        'f1':f1_score(y_test,prediction)}
    return results

In [155]:
def make_classification_plot(cm):
    disp = ConfusionMatrixDisplay(cm)
    disp = disp.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')
    plt.grid(False)
    plt.show()

In [156]:
results = {}

In [157]:
df['connectionTime'].describe()

/var/folders/7p/8d9trt0n3szgfv_nhh5rk6v00000gn/T/ipykernel_27169/3721555578.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df['connectionTime'].describe()


count                         31125
unique                        31081
top       2018-08-28 15:54:35+00:00
freq                              8
first     2018-05-01 13:23:18+00:00
last      2021-09-13 06:31:02+00:00
Name: connectionTime, dtype: object

In [158]:
df['spaceID'].value_counts()

CA-303      1792
CA-305      1399
CA-307      1136
11900388    1126
CA-315      1086
CA-311      1042
CA-309      1022
CA-304       994
CA-327       932
CA-313       908
CA-489       892
CA-321       862
CA-317       846
CA-325       820
CA-306       789
CA-323       759
CA-319       737
CA-314       694
CA-492       683
CA-312       665
CA-326       645
CA-320       615
CA-308       612
CA-490       587
CA-324       581
CA-322       578
CA-316       567
CA-310       561
CA-318       558
CA-494       533
CA-493       529
CA-491       506
CA-498       503
CA-495       458
CA-499       452
CA-496       452
CA-500       432
CA-497       396
CA-502       332
CA-501       330
CA-503       247
CA-212       200
CA-512       175
CA-504       149
CA-510       136
CA-506       124
CA-508       120
CA-513       108
CA-511       104
CA-507        90
CA-509        88
CA-149        76
CA-505        74
CA-213        15
CA-148         8
Name: spaceID, dtype: int64

In [159]:
tmp = df.copy().set_index('connectionTime')
tmp = tmp[tmp['spaceID'] == 'CA-303'].sort_index()

In [160]:
y = pd.DataFrame(index=pd.date_range('2018-05-1','2021-09-13', inclusive='both', freq='h', tz=0),columns=['is_available','sessionID'])
y['is_available'] = 1
for i in range(len(tmp)):
    start_ = tmp.index[i]
    end_ = tmp.loc[start_,'disconnectTime'] 
    session_ = tmp.loc[start_,'sessionID']
    y.loc[start_:end_,['is_available','sessionID']] = 0, session_

In [161]:
y.head()

,is_available,sessionID
2018-05-01 00:00:00+00:00,1,NaN
2018-05-01 01:00:00+00:00,1,NaN
2018-05-01 02:00:00+00:00,1,NaN
2018-05-01 03:00:00+00:00,1,NaN
2018-05-01 04:00:00+00:00,1,NaN


In [162]:
y.drop(columns='sessionID', inplace=True)

In [163]:
y['is_available'].value_counts()

1    20671
0     8874
Name: is_available, dtype: int64

In [164]:
X = pd.DataFrame(index=pd.date_range('2018-05-1','2021-09-13', inclusive='both', freq='h', tz=0),columns=['dow','hour','month'])
# X['dow'] = X.index.dt.hour
X['dow'] = X.index.dayofweek
X['hour'] = X.index.hour
X['month'] = X.index.month
X['connectionTime'] = X.index
X = holiday_processing(X).drop(columns=['connectionTime'])
X.head()

,dow,hour,month,is_holiday
2018-05-01 00:00:00+00:00,1,0,5,False
2018-05-01 01:00:00+00:00,1,1,5,False
2018-05-01 02:00:00+00:00,1,2,5,False
2018-05-01 03:00:00+00:00,1,3,5,False
2018-05-01 04:00:00+00:00,1,4,5,False


In [165]:
print(f'charger CA-303 is available {np.round(y.mean()*100,3)}% of the time')

charger CA-303 is available is_available    69.964
dtype: float64% of the time


In [166]:
X.shape[0] == y.shape[0]

True

# LSTM

In [168]:
# Create hold out test set
bin_numbers = pd.qcut(y['is_available'], q=10, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
print(f'the training data has an average availability of {np.round(y_train.mean()*100,3)}%')

the training data has an average availability of is_available    69.652
dtype: float64%


In [169]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
tf.random.set_seed(42)

In [170]:
#scaler = MinMaxScaler(feature_range=(0,1))
look_back = 1

In [171]:
model = Sequential()
model.add(LSTM(4, input_shape=(1,4)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [172]:
X_train = np.reshape(X_train.values.astype(float), (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values.astype(float), (X_test.shape[0], 1, X_test.shape[1]))

In [173]:
model.fit(X_train, y_train, epochs=10, batch_size=8, verbose=2)

Epoch 1/10
2955/2955 - 8s - loss: 0.2394 - 8s/epoch - 3ms/step
Epoch 2/10
2955/2955 - 4s - loss: 0.1985 - 4s/epoch - 1ms/step
Epoch 3/10
2955/2955 - 4s - loss: 0.1979 - 4s/epoch - 1ms/step
Epoch 4/10
2955/2955 - 4s - loss: 0.1978 - 4s/epoch - 1ms/step
Epoch 5/10
2955/2955 - 5s - loss: 0.1975 - 5s/epoch - 2ms/step
Epoch 6/10
2955/2955 - 4s - loss: 0.1977 - 4s/epoch - 1ms/step
Epoch 7/10
2955/2955 - 4s - loss: 0.1974 - 4s/epoch - 1ms/step
Epoch 8/10
2955/2955 - 4s - loss: 0.1976 - 4s/epoch - 1ms/step
Epoch 9/10
2955/2955 - 4s - loss: 0.1974 - 4s/epoch - 1ms/step
Epoch 10/10
2955/2955 - 4s - loss: 0.1972 - 4s/epoch - 1ms/step


In [174]:
predictions = model.predict(X_test)

185/185 [==============================] - 1s 2ms/step


In [175]:
from sklearn.metrics import mean_squared_error

In [176]:
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')

Mean Squared Error: 0.19258506881264306
